In [1]:
import ssdeep
import pickle,os,sys,gc
import pandas as pd
# pd.set_option('display.max_columns', None)
import numpy as np
from tqdm import tqdm
from collections import Counter
import multiprocessing as mp
from multiprocessing import Process, Manager, Pool
import subprocess
from multiprocessing import Pool 
import time,datetime
from itertools import chain
import math
from operator import itemgetter
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
import functools
import itertools
import operator
from multiprocessing.dummy import Pool as ThreadPool
import os.path
from os import path
import pickle,os,sys,gc
import pandas as pd
import numpy as np
import ssdeep
import glob
from tqdm import tqdm
import functools,operator
from datetime import datetime
from sklearn.metrics import *
from sklearn.metrics.pairwise import *
# import matplotlib相關套件
import matplotlib.pyplot as plt
# import字型管理套件
from matplotlib.font_manager import FontProperties
from matplotlib import pyplot
from operator import itemgetter
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

#### 計算preprocess以前各isp各天的壓縮前後GB大小

In [30]:
from pathlib import Path


def get_folder_size(folder):
    return ByteSize(sum(file.stat().st_size for file in Path(folder).rglob('*')))


class ByteSize(int):

    _kB = 1024
    _suffixes = 'B', 'kB', 'MB', 'GB', 'PB'

    def __new__(cls, *args, **kwargs):
        return super().__new__(cls, *args, **kwargs)

    def __init__(self, *args, **kwargs):
        self.bytes = self.B = int(self)
        self.kilobytes = self.kB = self / self._kB**1
        self.megabytes = self.MB = self / self._kB**2
        self.gigabytes = self.GB = self / self._kB**3
        self.petabytes = self.PB = self / self._kB**4
        *suffixes, last = self._suffixes
        suffix = next((
            suffix
            for suffix in suffixes
            if 1 < getattr(self, suffix) < self._kB
        ), last)
        self.readable = suffix, getattr(self, suffix)

        super().__init__()

    def __str__(self):
        return self.__format__('.2f')

    def __repr__(self):
        return '{}({})'.format(self.__class__.__name__, super().__repr__())

    def __format__(self, format_spec):
        suffix, val = self.readable
        return '{val:{fmt}} {suf}'.format(val=val, fmt=format_spec, suf=suffix)

    def __sub__(self, other):
        return self.__class__(super().__sub__(other))

    def __add__(self, other):
        return self.__class__(super().__add__(other))

    def __mul__(self, other):
        return self.__class__(super().__mul__(other))

    def __rsub__(self, other):
        return self.__class__(super().__sub__(other))

    def __radd__(self, other):
        return self.__class__(super().__add__(other))

    def __rmul__(self, other):
        return self.__class__(super().__rmul__(other))   

In [99]:
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
size_df = pd.DataFrame(columns=isp_li,index=date_li)
#壓縮前
root_path = '/home/antslab/RAID5/pcap/'
for da in date_li:
    date = str(da[:4])+'_'+str(da[4:6])+'_'+str(da[6:])
    for isp in isp_li:
        path = root_path+date+'/'+isp
        size = get_folder_size(path).GB
        size_df.loc[da,isp]=[size]
#壓縮後
root_path = '/home/antslab/NAS2_RAID6/pcap_process/'
for da in date_li:
    date = str(da[:4])+'_'+str(da[4:6])+'_'+str(da[6:])
    for isp in isp_li:
        path = root_path+date+'/'+isp
        size = get_folder_size(path).GB
        temp = size_df.loc[da,isp]
        temp.append(size)
        size_df.loc[da,isp]=temp
size_df
#         break

,中嘉寬頻,亞太電信,台固媒體,台灣固網,台灣大哥大,中華電信,凱擘,台灣之星,台灣基礎開發,台灣碩網
20200106,"[22.9977661203593, 94.69588289503008]","[11.682307879440486, 37.36275399196893]","[22.485208248719573, 124.34045305475593]","[25.054296598769724, 66.93216721434146]","[33.00308726541698, 82.26698496565223]","[46.438600975088775, 117.85005235578865]","[34.02302599232644, 83.04273752588779]","[3.1792144775390625, 8.253010737709701]","[19.36416457220912, 49.71311922650784]","[28.666001605801284, 115.56446789298207]"
20200107,"[21.400398281402886, 87.80454102158546]","[12.337482975795865, 39.94172821845859]","[29.80459010694176, 170.72866547852755]","[33.87399543169886, 89.95072550699115]","[24.285831147804856, 59.291690402664244]","[41.82352768816054, 106.5314063038677]","[46.96689920593053, 114.8009220128879]","[54.98275569267571, 143.57253547944129]","[21.121206305921078, 54.092634295113385]","[36.21226442139596, 141.89316245168447]"
20200108,"[18.516566307283938, 73.21716434694827]","[10.900887260213494, 34.39156724046916]","[28.179426619783044, 159.77352996822447]","[31.078676293604076, 81.09213962592185]","[0.1722869873046875, 0.43036659248173237]","[39.136319869197905, 98.81970036868006]","[46.09693148266524, 112.31819145195186]","[51.110973755829036, 132.40112011693418]","[15.02602611668408, 38.331466518342495]","[43.00054021272808, 162.41746459063143]"
20200109,"[19.004350412636995, 71.81381406541914]","[10.911588756367564, 33.09088975470513]","[25.91101478319615, 145.05681699328125]","[16.734383570961654, 44.895547543652356]","[0.941925048828125, 2.3111567478626966]","[30.966476498171687, 78.92627032566816]","[29.39612957276404, 71.07273649610579]","[48.85265841521323, 126.47137834876776]","[20.662016907706857, 54.236665327101946]","[38.21850425377488, 148.69663755968213]"
20200110,"[2.2979812622070312, 8.814227112568915]","[4.471859212964773, 13.271716803312302]","[38.57341337390244, 228.163732082583]","[16.996942220255733, 44.395290930755436]","[1.808837890625, 4.5076248943805695]","[53.561792416498065, 138.27866089344025]","[37.67561180423945, 93.10983189288527]","[57.469349436461926, 149.010758231394]","[20.616785728372633, 53.20092592481524]","[45.42625961918384, 178.05772034078836]"
20200111,"[1.0161170959472656, 4.041122065857053]","[8.450147309340537, 27.526308899745345]","[59.78928914666176, 370.60446255095303]","[13.218987796455622, 32.85034523997456]","[15.607192993164062, 38.19045589957386]","[54.94324762746692, 142.4895850243047]","[24.52468352485448, 59.829949918203056]","[58.20305017475039, 150.69895350188017]","[20.52224489208311, 53.226023332215846]","[45.53947496693581, 184.94426775816828]"
20200112,"[12.831710815429688, 49.661712939850986]","[10.050270940177143, 31.0974273243919]","[55.299916822463274, 334.71838948503137]","[17.39263062737882, 43.46204193588346]","[20.167719746008515, 48.11477105598897]","[43.33650591317564, 112.32671623490751]","[20.25234789121896, 51.524440039880574]","[44.26231833733618, 113.4701829040423]","[16.822678077965975, 42.91725635249168]","[42.166008131578565, 162.0279680956155]"


In [109]:
pickle.dump(obj=size_df,file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/before_after_pcapfilesize_df.pkl','wb'))

In [105]:
# calculate every day size before decompress
trans_size_df = size_df.T
total_count1 = 0
total_count2 = 0
for da in date_li:
    temp = trans_size_df[da].tolist()
    count1 = 0
    count2 = 0
    for t in temp:
        count1 = count1+t[0] #before
        count2 = count2+t[1] #after
    total_count1 = total_count1 + count1
    total_count2 = total_count2 + count2
    print("Before decompress Size Total",da,':',count1) #/len(isp_li)
    print("After decompress Size Total",da,':',count2) #/len(isp_li)
print("Total Before",total_count1)
print("TOtal After",total_count2)
# def calc_day_before()
# calculate every isp size before/after decompress
total_count1 = 0
total_count2 = 0
for isp in isp_li:
    temp = size_df[isp].tolist()
    count1 = 0
    count2 = 0
    for t in temp:
        count1 = count1+t[0] #before
        count2 = count2+t[1] #after
    total_count1 = total_count1 + count1
    total_count2 = total_count2 + count2
    print("Before decompress Size Total",isp,':',count1) #/len(isp_li)
    print("After decompress Size Total",isp,':',count2) #/len(isp_li)
print("Total Before",total_count1)
print("TOtal After",total_count2)

Before decompress Size Total 20200106 : 246.89367373567075
After decompress Size Total 20200106 : 780.0216298606247
Before decompress Size Total 20200107 : 322.80895125772804
After decompress Size Total 20200107 : 1008.6080111712217
Before decompress Size Total 20200108 : 283.2186349052936
After decompress Size Total 20200108 : 893.1927108205855
Before decompress Size Total 20200109 : 241.59904821962118
After decompress Size Total 20200109 : 776.5719131622463
Before decompress Size Total 20200110 : 278.8988329647109
After decompress Size Total 20200110 : 910.8104891069233
Before decompress Size Total 20200111 : 301.81443552765995
After decompress Size Total 20200111 : 1064.401474190876
Before decompress Size Total 20200112 : 282.58210730273277
After decompress Size Total 20200112 : 989.3209063680843
Total Before 1957.8156839134172
TOtal After 6422.927134680562
Before decompress Size Total 中嘉寬頻 : 98.0648902952671
After decompress Size Total 中嘉寬頻 : 390.0484644472599
Before decompress Siz

In [32]:
#計算總hp的壓縮檔案大小
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
# size_df = pd.DataFrame(columns=isp_li,index=date_li)
total_size = 0
#壓縮前
root_path = '/home/antslab/RAID5/honeypot/'
for da in date_li:
    date = str(da[:4])+'_'+str(da[4:6])+'_'+str(da[6:])
    path = root_path+date+'/'
    size = get_folder_size(path).GB
    print(da,size)
    total_size = total_size + size
total_size

OSError: [Errno 116] Stale file handle: '/home/antslab/RAID5/honeypot/2020_01_06'

#### 計算session/packets數量

In [ ]:
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
# protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']
# take_columns = ['session_i_tt_packet','session_o_tt_packet']
pkt_sess_df = pd.DataFrame(columns=isp_li,index=date_li)
country_count_dict = {}
protocol_count_dict = {}
for da in tqdm(date_li):
    for isp in isp_li:
        print("===Now is about to run:",da,isp,"===")
        gc.collect()
        dir_path = "/mnt/Raid160TB/pcap_inter/"+str(da[:4])+'_'+str(da[4:6])+'_'+str(da[6:])+"/"+isp+"/intermeidate_data/"
        saved_path = dir_path+"df2_tuples_"+isp+".pkl"
        df2,pickle_dir,picture_dir = pickle.load(open(saved_path,'rb'))  
        
        inbond_pkt = df2.session_i_tt_packet.sum()
        outbond_pkt = df2.session_o_tt_packet.sum()
        session = len(df2)
        pkt_sess_df.loc[da,isp]=[inbond_pkt,outbond_pkt,session]
        
        country_session_count = dict(df2.country.value_counts())
        for k,v in country_session_count.items():
            try:
                temp = country_count_dict[k]
                temp.append(v)
                country_count_dict[k]=temp
            except KeyError:
                country_count_dict[k] = [v]
                
        protocol_session_count = dict(df2.frame_i_max_protocols.value_counts())
        for k,v in protocol_session_count.items():
            try:
                temp = protocol_count_dict[k]
                temp.append(v)
                protocol_count_dict[k]=temp
            except KeyError:
                protocol_count_dict[k] = [v]  
        pickle.dump(obj=(pkt_sess_df,country_count_dict,protocol_count_dict),file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/before_preprocess_stat_3tupleaspect.pkl','wb'))

#         break
#     break

pickle.dump(obj=(pkt_sess_df,country_count_dict,protocol_count_dict),file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/before_preprocess_stat_3tupleaspect.pkl','wb'))
pkt_sess_df

 29%|██▊       | 2/7 [3:53:19<10:43:31, 7722.20s/it]

In [7]:
trans_pkt_sess_df = pkt_sess_df.T

# calculate every day pkts/sess 
trans_pkt_sess_df = pkt_sess_df.T
total_count1 = 0
total_count2 = 0
total_count3 = 0
for da in date_li:
    temp = trans_pkt_sess_df[da].tolist()
    count1 = 0
    count2 = 0
    count3 = 0
    for t in temp:
        count1 = count1+t[0] #inbond#
        count2 = count2+t[1] #outbond#
        count3 = count3+t[2] #session#
    total_count1 = total_count1 + count1
    total_count2 = total_count2 + count2
    total_count3 = total_count3 + count3
    print("In-bond Packets Total#",da,':',count1) #/len(isp_li)
#     print("Out-bond Packets Total#",da,':',count2) #/len(isp_li)
    print("Total Packets#",da,':',count1+count2)
    print("Sessions Total#",da,':',count3) #/len(isp_li)
print("Total In-bond Packets#:",total_count1)
# print("Total Out-bond Packets#:",total_count2)
print("Total Packets#:",total_count2+total_count1)
print("Total Sessions#:",total_count3)
# def calc_day_before()
# calculate every isp size before/after decompress
total_count1 = 0
total_count2 = 0
total_count3 = 0
for isp in isp_li:
    temp = pkt_sess_df[isp].tolist()
    count1 = 0
    count2 = 0
    count3 = 0
    for t in temp:
        count1 = count1+t[0] #inbond#
        count2 = count2+t[1] #outbond#
        count3 = count3+t[2] #session#
    total_count1 = total_count1 + count1
    total_count2 = total_count2 + count2
    total_count3 = total_count3 + count3    
    print("In-bond Packets Total#",isp,':',count1) #/len(isp_li)
#     print("Out-bond Packets Total#",isp,':',count2) #/len(isp_li)
    print("Total Packets#",isp,':',count1+count2)
    print("Sessions Total#",isp,':',count3) #/len(isp_li)
print("Total In-bond Packets#:",total_count1)
# print("Total Out-bond Packets#:",total_count2)
print("Total Packets#:",total_count2+total_count1)
print("Total Sessions#:",total_count3)


# pkt_sess_df.T #[inbond_pkt,outbond_pkt,session]

In-bond Packets Total# 20200106 : 711043808
Total Packets# 20200106 : 1356516470
Sessions Total# 20200106 : 27509059
In-bond Packets Total# 20200107 : 916171619
Total Packets# 20200107 : 1762961034
Sessions Total# 20200107 : 36325031
In-bond Packets Total# 20200108 : 685161743
Total Packets# 20200108 : 1320526805
Sessions Total# 20200108 : 25367306
In-bond Packets Total# 20200109 : 647477689
Total Packets# 20200109 : 1211713225
Sessions Total# 20200109 : 26835872
In-bond Packets Total# 20200110 : 775596350
Total Packets# 20200110 : 1493623570
Sessions Total# 20200110 : 25249322
In-bond Packets Total# 20200111 : 883201576
Total Packets# 20200111 : 1723602924
Sessions Total# 20200111 : 22069515
In-bond Packets Total# 20200112 : 885263390
Total Packets# 20200112 : 1719926235
Sessions Total# 20200112 : 33747926
Total In-bond Packets#: 5503916175
Total Packets#: 10588870263
Total Sessions#: 197104031
In-bond Packets Total# 中嘉寬頻 : 471754017
Total Packets# 中嘉寬頻 : 837075450
Sessions Total# 中嘉寬

In [22]:
def sum_sess(li):
    return sum(li)
protocol_df = pd.DataFrame(protocol_count_dict.items(),columns=['protocol','session_num'])
protocol_df['session_count'] = protocol_df['session_num'].map(sum_sess)
protocol_df = protocol_df.sort_values(['session_count'],ascending=False)
protocol_df['portion(%)'] = (protocol_df['session_count']/protocol_df['session_count'].sum())*100
protocol_df.head(60)

,protocol,session_num,session_count,portion(%)
0,eth:ethertype:ip:tcp:tds:gss-api,"[1220812, 1655239, 1316268, 1060242, 1497885, ...",73178320,37.126750
1,eth:ethertype:ip:tcp:ssh,"[1174527, 49289, 668210, 1223636, 1977670, 272...",48158748,24.433162
2,eth:ethertype:ip:tcp:tds,"[860395, 721051, 1030042, 628514, 878800, 6788...",46317723,23.499125
4,eth:ethertype:ip:tcp:nbss:smb,"[100298, 264839, 132428, 221342, 204007, 29624...",9827147,4.985767
5,eth:ethertype:ip:tcp:http,"[71039, 90610, 64108, 174582, 175963, 149824, ...",7706548,3.909889
3,eth:ethertype:ip:tcp:data,"[178466, 44477, 52516, 39262, 47934, 43412, 44...",3048682,1.546738
7,eth:ethertype:ip:tcp:nbss:smb:gss-api:spnego:n...,"[15268, 16467, 20061, 17369, 28243, 48222, 277...",2176272,1.104124
24,eth:ethertype:ip:tcp:telnet,"[138, 105135, 111, 428, 445, 291, 219, 24, 160...",1772070,0.899053
8,eth:ethertype:ip:tcp:http:urlencoded-form,"[10877, 3064, 8814, 21344, 21575, 31715, 6908,...",773529,0.392447
10,eth:ethertype:ip:tcp:smtp,"[6529, 13873, 6383, 21179, 15593, 12294, 7988,...",718089,0.364320


In [28]:
total_session_num = 0
total_percent = 0
protocols_need = ['ssh','smb','http','smtp','telnet','ftp','pop','imap','mysql','sip','rpc']
for proto in protocols_need:
    temp = protocol_df[protocol_df.protocol.str.contains(proto)]
    session_num = temp['session_count'].sum()
    portion = temp['portion(%)'].sum()
    total_session_num = total_session_num + session_num
    total_percent = total_percent + portion
    print(proto,session_num,portion)
print("TOTAL:",total_session_num,total_percent)

ssh 48163041 24.43534044212419
smb 12955813 6.573083733635055
http 8694422 4.411082795156026
smtp 724842 0.36774590368473997
telnet 1772070 0.8990531502625637
ftp 253832 0.1287807249360618
pop 52414 0.026592048744046234
imap 27546 0.013975361061996748
mysql 170963 0.08673744475575945
sip 176376 0.08948371025451023
rpc 157109 0.07970866917480751
TOTAL: 73148428 37.11158398378977


In [25]:
protocol_df.session_count.sum()

197104031

In [34]:
df2.columns

Index(['session_time', 'session_time_list', 'session_duration',
       'session_i_tt_packet', 'session_o_tt_packet',
       'session_i_tt_frame_length', 'session_o_tt_frame_length',
       'udp_i_tt_length', 'udp_o_tt_length', 'udp_i_avg_length',
       'udp_o_avg_length', 'icmp_i_avg_length', 'icmp_o_avg_length',
       'icmp_i_tt_datagram_length', 'icmp_o_tt_datagram_length',
       'tcp_i_tt_payload_length', 'tcp_o_tt_payload_length',
       'tcp_i_avg_payload_length', 'tcp_o_avg_payload_length', 'ip_src',
       'ip_dst', 'tcp_srcport', 'tcp_dstport', 'country', 'isp', 'domain',
       'frame_i_max_protocols', 'frame_o_max_protocols', 'tcp_i_payload_list',
       'tcp_o_payload_list'],
      dtype='object')

In [35]:
df2['tcp_o_tt_payload_length']

0                   489
1                   364
2                   232
3                   232
4                   232
                 ...   
962072706589         10
962072706590        232
962072706591    2088016
962072706592        364
962072706593        232
Name: tcp_o_tt_payload_length, Length: 3240052, dtype: int32

In [38]:
temp = df2[df2.frame_i_max_protocols.str.contains('http')]
temp.head(60)

,session_time,session_time_list,session_duration,session_i_tt_packet,session_o_tt_packet,session_i_tt_frame_length,session_o_tt_frame_length,udp_i_tt_length,udp_o_tt_length,udp_i_avg_length,...,ip_dst,tcp_srcport,tcp_dstport,country,isp,domain,frame_i_max_protocols,frame_o_max_protocols,tcp_i_payload_list,tcp_o_payload_list
194,1.578785e+09,"[1578784906.288008, 1578784906.288278, 1578784...",0.812252,36,24,16375,46480,NaN,NaN,NaN,...,219.84.227.172,60803,80,China,None,None,eth:ethertype:ip:tcp:http:data,eth:ethertype:ip:tcp:http:data-text-lines,[[24:dAawYDQxLrTXQtcJcS18YH+w18YHxVoH9HdlUX18Y...,[[48:KTgb435ECp4tOv2QxEsf9vOttqUhzfBnsnIzfUf4B...
206,1.578838e+09,"[1578838343.396999, 1578838343.397207, 1578838...",15.105635,6,4,620,2897,NaN,NaN,NaN,...,219.84.227.43,35307,80,Hong Kong,None,None,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data-text-lines:http...,[[12:7KS/cHnMQ7s95HfxjmHXq/Is0Kj51h6Y1awh5UP+/...,[[96:3n1MCp42QE8QMp0cgO0qkA3A8ZuBntn5WnblTVy4G...
207,1.578838e+09,"[1578837830.570376, 1578837830.57065, 15788378...",15.114566,6,4,619,2891,NaN,NaN,NaN,...,219.84.227.43,57659,80,Hong Kong,None,None,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data-text-lines:http...,[[12:7KS/cHnMQ7s95Hf+3qjmHXq/Is0Kj51h6Y1awh5UP...,[[96:3n1MCp42QE8QMp0cgO0qkA3A8ZuBntn5WnblTVy4G...
208,1.578844e+09,"[1578844098.071577, 1578844098.071864, 1578844...",17.966524,13,6,2158,3029,NaN,NaN,NaN,...,219.84.227.43,64682,80,Hong Kong,None,None,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data-text-lines:http...,[[12:7KS/cHnMQ7s95HfmEFjmHXq/Is0Kj51h6Y1awh5UP...,[[96:3n1MCp42QE8QMp0cgO0qkA3A8ZuBntn5WnblTVy4G...
268,1.578796e+09,"[1578796368.324438, 1578796368.324817, 1578796...",7.831007,7,4,658,2891,NaN,NaN,NaN,...,219.84.227.10,35989,80,China,None,None,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data-text-lines,[[12:7KS/cHnMQ7s95HfWfUTf5jmHXq/IsheY1awh5UPOn...,[[96:3n1MCp42QE8QMp0cgO0qkA3A8ZuBntn5WnblTVy4G...
292,1.578813e+09,"[1578812861.649316, 1578812861.649661, 1578812...",0.755571,5,5,664,778,NaN,NaN,NaN,...,219.84.227.128,33974,80,Thailand,None,3bb.co.th,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data,[[24:tALHy8OBBBBBBBBBBB1owbAs+rX+rQ+rd+r2+rL+r...,[[24:KtoUNnzxT4BlDtdbNesV2K14UeXoUNnzxT4Bll:Kt...
293,1.578813e+09,"[1578813096.163151, 1578813096.163404, 1578813...",0.690489,6,4,1114,712,NaN,NaN,NaN,...,219.84.227.128,40484,80,Thailand,None,3bb.co.th,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data,[[48:+TR1FVgnGt0b6R/4OVIPiZcDe8rBfIua8:+TDgnGt...,[[24:KtoUNnzxT4BlDtdbNesV2K1OiioUNnzxT4Bll:Kt1...
295,1.578823e+09,"[1578823102.53742, 1578823102.537766, 15788231...",14.500953,6,3,609,2843,NaN,NaN,NaN,...,219.84.227.53,11266,80,China,None,online.tj.cn,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data-text-lines,[[12:7KS/cHnMQ7s95HfijmHXq/IYNb6Y1awh5UPkV8U2V...,[[96:3n1MCp42QE8QMp0cgO0qkA3A8ZuBntn5WnblTVy4G...
296,1.578824e+09,"[1578824157.050113, 1578824157.05028, 15788241...",16.357607,6,4,609,2891,NaN,NaN,NaN,...,219.84.227.53,22002,80,China,None,online.tj.cn,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data-text-lines:http...,[[12:7KS/cHnMQ7s95Hf/oqjmHXq/IYNb6Y1awh5UPkV8U...,[[96:3n1MCp42QE8QMp0cgO0qkA3A8ZuBntn5WnblTVy4G...
297,1.578829e+09,"[1578829337.63346, 1578829337.633683, 15788293...",11.481255,8,6,722,3017,NaN,NaN,NaN,...,219.84.227.53,34961,80,China,None,online.tj.cn,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data-text-lines:http...,[[12:7KS/cHnMQ7s95Hf9cjyK8jmHXq/IYNb6Y1awh5UPk...,[[96:3n1MCp42QE8QMp0cgO0qkA3A8ZuBntn5WnblTVy4G...


In [46]:
42270+1985820

2028090

In [50]:
# temp.loc[1621,'session_time']
# temp.loc[1621]#,'session_time']
takeidx = 2354
print("start_time:",temp.loc[takeidx,'session_time'])
print("duration:",temp.loc[takeidx,'session_duration'])
print("#pkt:",temp.loc[takeidx,'session_i_tt_packet']+temp.loc[takeidx,'session_o_tt_packet'])
print("ip.src:",temp.loc[takeidx,'ip_src'])
print("ip.dst:",temp.loc[takeidx,'ip_dst'])
print("srcport:",temp.loc[takeidx,'tcp_srcport'])
print("dstport:",temp.loc[takeidx,'tcp_dstport'])
print("frame.length:",temp.loc[takeidx,'session_i_tt_frame_length']+temp.loc[takeidx,'session_o_tt_frame_length'])
print("payload.length:",temp.loc[takeidx,'tcp_i_tt_payload_length']+temp.loc[takeidx,'tcp_o_tt_payload_length'])


start_time: 1578838444.984012
duration: 108.02627
#pkt: 16
ip.src: 173.172.100.234
ip.dst: 219.84.227.183
srcport: 34847
dstport: 80
frame.length: 16992
payload.length: 16100


In [33]:
df2.frame_i_max_protocols.str.contains('http')

,session_time,session_time_list,session_duration,session_i_tt_packet,session_o_tt_packet,session_i_tt_frame_length,session_o_tt_frame_length,udp_i_tt_length,udp_o_tt_length,udp_i_avg_length,...,ip_dst,tcp_srcport,tcp_dstport,country,isp,domain,frame_i_max_protocols,frame_o_max_protocols,tcp_i_payload_list,tcp_o_payload_list
0,1.578834e+09,"[1578834253.498932, 1578834253.499304, 1578834...",5.646473,9,12,965,1185,NaN,NaN,NaN,...,61.64.230.87,49578,445,Taiwan,None,hinet.net,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[[6:iCV00dYMe9hNV9oqKc5EOOfoa0VfCy/a9evgfey9FV...
1,1.578818e+09,"[1578817746.899339, 1578817746.89951, 15788177...",0.870590,8,11,899,1006,NaN,NaN,NaN,...,219.84.227.99,58074,445,Taiwan,None,hinet.net,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[[6:iCV00dYMe9hNV9ocN3rIfoa0VfCy/a9evgfey9FV1R...
2,1.578804e+09,"[1578803808.314863, 1578803808.315053, 1578803...",15.393650,8,10,808,820,NaN,NaN,NaN,...,61.62.173.116,12946,445,Taiwan,None,hinet.net,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[3:11QRfDp03vV1V1Ud2FV1V1V1V1V1V1V1V1V1V1V1V1...,[[3:11QQoW503vV1V1cdcV1V1V1V1V1V1V1V1V1V133FfX...
3,1.578838e+09,"[1578838439.314418, 1578838439.31466, 15788384...",8.585497,6,8,700,700,NaN,NaN,NaN,...,61.64.230.110,62495,445,Thailand,None,totisp.net,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[3:11QRfDp03vV1V1Ud2FV1V1V1V1V1V1V1V1V1V1V1V1...,[[3:11QQoW503vV1V1cdcV1V1V1V1V1V1V1V1V1V133FfX...
4,1.578796e+09,"[1578796448.517556, 1578796448.517701, 1578796...",2.843800,7,8,842,700,NaN,NaN,NaN,...,61.64.230.121,50401,445,Thailand,None,totinternet.net,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[3:11QRfDp03vV1V1Ud2FV1V1V1V1V1V1V1V1V1V1V1V1...,[[3:11QQoW503vV1V1cdcV1V1V1V1V1V1V1V1V1V133FfX...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962072706589,1.578844e+09,"[1578843857.835699, 1578843857.835907, 1578843...",170.159576,6,12,409,806,NaN,NaN,NaN,...,219.84.227.247,59386,23,Turkey,None,None,eth:ethertype:ip:tcp:telnet,eth:ethertype:ip:tcp:telnet,"[[3:3Dt:B, 1578843869.154793000, 3], [3:kfTDn8...","[[3:3Dpa7ukTcC/V:07T9d, 1578843868.814361000, ..."
962072706590,1.578765e+09,"[1578765149.554278, 1578765149.554523, 1578765...",3.851458,6,8,700,700,NaN,NaN,NaN,...,219.84.227.97,50614,445,Russia,None,None,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[3:11QRfDp03vV1V1Ud2FV1V1V1V1V1V1V1V1V1V1V1V1...,[[3:11QQoW503vV1V1cdcV1V1V1V1V1V1V1V1V1V133FfX...
962072706591,1.578765e+09,"[1578765292.490625, 1578765292.490853, 1578765...",2.950797,520,439,31769,2116998,NaN,NaN,NaN,...,219.84.227.184,53556,80,United States,None,linode.com,eth:ethertype:ip:tcp:http,eth:ethertype:ip:tcp:http:data-text-lines,[[12:rHXq6xcjuDGIs0+R56Y1awh5UP+/4D7eiMJNkZr1N...,[[96:Zb4aCp4QuQxr9+jqozJnsnSnrnb0qB4DZfT28:d+p...
962072706592,1.578830e+09,"[1578829917.264741, 1578829917.265004, 1578829...",3.774506,8,11,899,1006,NaN,NaN,NaN,...,219.84.227.106,51627,445,United States,None,rr.com,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[[6:iCV00dYMe9hNV9oSRiU/ucQfoa0VfCy/a9evgfey9F...


In [3]:
one,two,three = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/before_preprocess_stat_3tupleaspect.pkl','rb'))
one

,中嘉寬頻,亞太電信,台固媒體,台灣固網,台灣大哥大,中華電信,凱擘,台灣之星,台灣基礎開發,台灣碩網
20200106,"[110067864, 84498249, 3688364]","[29261261, 24385052, 3012973]","[86211044, 99150304, 3339508]","[71208117, 64464161, 3463789]","[101000280, 92178410, 4913920]","[112564591, 84188892, 3316952]","[63541700, 75807387, 3073673]","[8803253, 6894441, 428064]","[50557054, 42980805, 1763750]","[77828644, 70924961, 508066]"
20200107,"[102077135, 83227713, 3932166]","[35661793, 30093356, 3883244]","[110302245, 130692433, 3461682]","[96056981, 88495391, 4286177]","[80116652, 74809299, 4281127]","[102578872, 80168459, 4090062]","[79109763, 97038713, 3204305]","[159612583, 124222059, 5454381]","[51338968, 44447925, 1701162]","[99316627, 93594067, 2030725]"
20200108,"[93288935, 72786931, 3935529]","[14205344, 12062888, 1392756]","[103910477, 122642262, 3390532]","[90746672, 86016508, 3512094]","[380135, 356866, 18545]","[90791970, 71602897, 3031452]","[78518253, 94061126, 3151242]","[146605188, 114957892, 4565637]","[36922508, 32624058, 1579390]","[29792261, 28253634, 790129]"
20200109,"[96788040, 73794293, 4067492]","[26529218, 21225212, 2989593]","[99197328, 114393581, 3562766]","[46050371, 40822733, 3064024]","[2525913, 2396810, 172374]","[73577643, 57698883, 3034011]","[3597889, 3222038, 351277]","[142841049, 111959774, 5452447]","[49265176, 38861606, 1441963]","[107105062, 99860606, 2699925]"
20200110,"[9944555, 7031100, 335154]","[7029606, 5166997, 545394]","[132451509, 174443650, 3526213]","[52998683, 50399083, 3159992]","[4874264, 4602728, 288283]","[161691087, 93480788, 3298057]","[68950572, 85508590, 3295531]","[166644083, 136126016, 6399851]","[48304707, 41045408, 1511276]","[122707284, 120222860, 2889571]"
20200111,"[4398774, 2986215, 98382]","[22173387, 18807748, 1719824]","[200426977, 267865787, 2500495]","[53423923, 54523932, 4079429]","[45021939, 43435172, 1895500]","[165344926, 95678558, 2207256]","[50010401, 61175320, 2367702]","[162174141, 130442483, 4271076]","[50530243, 41347325, 1121797]","[129696865, 124138808, 1808054]"
20200112,"[55188714, 40996932, 1755005]","[33867831, 28476634, 4459596]","[194309369, 249670808, 3943532]","[69088109, 67411159, 4549545]","[71401833, 69665435, 3993249]","[121148294, 72013454, 706560]","[37760715, 41879569, 2539162]","[140645839, 115260330, 6969695]","[44465941, 36000864, 1591530]","[117386745, 113287660, 3240052]"


In [115]:
# df2.session_i_tt_packet.sum()

# pkt_sess_df = pd.DataFrame(columns=isp_li,index=date_li)
# country_session_count = dict(df2.country.value_counts())
# country_count_dict ={}
# for k,v in country_session_count.items():
#     try:
#         temp = country_count_dict[k]
#         temp.append(v)
#         country_count_dict[k]=temp
#     except KeyError:
#         country_count_dict[k] = [v]
# country_count_dict
# # dict(df2.frame_i_max_protocols.value_counts())
# # pkt_sess_df

# protocol_count_dict= {}
# inbond_pkt = df2.session_i_tt_packet.sum()
# outbond_pkt = df2.session_o_tt_packet.sum()
# session = len(df2)
# pkt_sess_df.loc[da,isp]=[inbond_pkt,outbond_pkt,session]

# country_session_count = dict(df2.country.value_counts())
# for k,v in country_session_count.items():
#     try:
#         temp = country_count_dict[k]
#         temp.append(v)
#         country_count_dict[k]=temp
#     except KeyError:
#         country_count_dict[k] = [v]

# protocol_session_count = dict(df2.frame_i_max_protocols.value_counts())
# for k,v in protocol_session_count.items():
#     try:
#         temp = protocol_count_dict[k]
#         temp.append(v)
#         protocol_count_dict[k]=temp
#     except KeyError:
#         protocol_count_dict[k] = [v]     

# country_count_dict

# pkt_sess_df = pd.DataFrame(columns=isp_li,index=date_li)
# df2.columns

110067864

#### 計算封包數量

In [6]:
# date_li = ['20200101','20200102','20200103','20200104','20200105',
#            '20200113','20200114','20200115','20200116','20200117','20200118'] #改
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']
take_columns = ['session_i_tt_packet','session_o_tt_packet']
whole_dict = {}
for da in tqdm(date_li):
    for isp in isp_li:
        gc.collect()
        dir_path = "/mnt/Raid160TB/pcap_inter/"+str(da[:4])+'_'+str(da[4:6])+'_'+str(da[6:])+"/"+isp+"/intermeidate_data/"
        saved_path = dir_path+"df2_tuples_"+isp+".pkl"
        df2,pickle_dir,picture_dir = pickle.load(open(saved_path,'rb'))        
        for protocol_name in protocols_need:
            df_proto = df2[(df2.frame_i_max_protocols.str.contains(protocol_name))&(df2.frame_o_max_protocols.str.contains(protocol_name))]            
            for col in take_columns:
                name = da+'_'+isp+'_'+protocol_name+'_'+col
                whole_dict[name] = df_proto[col].tolist()
            
            
            
whole_dict.keys()

100%|██████████| 7/7 [74:47:34<00:00, 38464.88s/it]   


ValueError: arrays must all be same length

In [14]:
pickle.dump(obj=whole_dict,file=open("/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/intermeidate_data/count_df2_sessionpackets.pkl",'wb'))

依照isp計算封包數量

In [23]:
isp_packets = {}
all_keys = list(whole_dict.keys())
for isp in tqdm(isp_li):
    packet_count = 0
    for key in all_keys:
        if isp in key:
            packet_count = packet_count + sum(whole_dict[key])
    isp_packets[isp] = packet_count
temp = pd.DataFrame.from_dict(isp_packets,orient='index')
print(temp[0].sum())
temp

100%|██████████| 10/10 [00:00<00:00, 16.62it/s]

9214458093


,0
中嘉寬頻,699183335
亞太電信,152535669
台固媒體,1894641241
台灣固網,790546855
台灣大哥大,517958039
中華電信,1234400829
凱擘,728790977
台灣之星,1448951784
台灣基礎開發,523263781
台灣碩網,1224185583


依照protocol計算封包數量

In [21]:
# for proto in protocols_need:
proto_packets = {}
all_keys = list(whole_dict.keys())
for proto in tqdm(protocols_need):
    packet_count = 0
    for key in all_keys:
        if proto in key:
            packet_count = packet_count + sum(whole_dict[key])
    proto_packets[proto] = packet_count
temp = pd.DataFrame.from_dict(proto_packets,orient='index')
print(temp[0].sum())
temp

100%|██████████| 11/11 [00:00<00:00, 18.81it/s]

9214458093


,0
http,7098142157
mysql,2818401
ftp,6453332
smb,241820396
smtp,14782691
imap,380353
pop,942589
rpc,3449057
ssh,1806753753
telnet,36564835


依照不同電信商、protcol及日期反推封包數量

In [81]:
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']
proto_isp_date_packets = {}
all_keys = list(whole_dict.keys())
for proto in tqdm(protocols_need):
    for isp in isp_li:
        for date in date_li:
            counter = 0
            for key in all_keys:
                if (proto in key)and(isp in key)and(date in key):
                    counter = counter + sum(whole_dict[key])
            proto_isp_date_packets[proto+'_'+isp+'_'+date]=counter
packet_count = pd.DataFrame.from_dict(proto_isp_date_packets,orient='index')
packet_count.columns = ['packet_num']
# print(temp[0].sum())
packet_count                

100%|██████████| 11/11 [00:00<00:00, 15.99it/s]


,packet_num
http_中嘉寬頻_20200106,126364749
http_中嘉寬頻_20200107,111947097
http_中嘉寬頻_20200108,93768133
http_中嘉寬頻_20200109,91728334
http_中嘉寬頻_20200110,11266917
...,...
sip_台灣碩網_20200108,988
sip_台灣碩網_20200109,1983
sip_台灣碩網_20200110,337372
sip_台灣碩網_20200111,69562


### 計算AP的封包數量

In [82]:
pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/crossisp_signature_dfdict.pkl' #改
agg_proto_df=pickle.load(open(pickle_dir,'rb'))
agg_proto_df['http']

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,signature,ssdeep_clean,clusterkey_timelist,date_count,sum
0,20200108_8589990159_http,"[1578491422.648805, 1578491434.488182, 1578491...","[South Korea, South Korea, South Korea, South ...","[122.41.87.188, 122.41.87.188, 122.41.87.188, ...",[48:o0SP/FgTwfTihvZMlsSp0UUwaIqF6GY18nR:PSPdMw...,"[(37.5112, 126.9741), (37.5112, 126.9741), (37...",亞太電信,http,118,[48:o0SP/FgTwfTihvZMlsSp0UUwaIqF6GY18nR:PSPdMw...,"{'48': ['a', 'q', 'F6GY18n', 'R', 'M', '5Y1k']}",[48:Ga7UCqF6GY18nwiLkcRq09jcRq09uMRRdJtYMY8:hg...,"[1578491500.667118, 1578491500.667435, 1578491...","{'20200108': 7, '20200109': 111}",0.0
1,20200106_8589962872_http,"[1578244875.871224, 1578279973.587717, 1578291...","[Netherlands, Netherlands, Netherlands, Nether...","[51.15.71.134, 51.15.71.134, 51.15.71.134, 51....",[24:LepsAkMVAab5CYD0sVUYaP+shmGvQEjyOFEoW:Leps...,"[(52.3759, 4.8975), (52.3759, 4.8975), (52.375...",亞太電信,http,25,[24:LepsAkMVAab5CYD0sVUYaP+shmGvQEjyOFEoW:Leps...,{'12': ['Bhzeps+hkMVk3HXq6xgzhrHCYPEjyOFE4nBQo...,[12:Bhzeps+hkMVk3HXq6xgzhrHCYPEjyOFE4nBQom:Lep...,"[1578315077.277642, 1578315077.277914, 1578315...","{'20200106': 5, '20200107': 5, '20200108': 2, ...",0.0
2,20200106_206158498684_http,"[1578325476.581104, 1578325480.685079]","[Canada, Canada]","[209.99.129.253, 209.99.129.253]",[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,"[(43.6655, -79.4204), (43.6655, -79.4204)]",亞太電信,http,2,[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,{'96': ['rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41u...,[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,"[1578325476.581104, 1578325476.581368, 1578325...",{'20200106': 2},0.0
3,20200109_128849077624_http,"[1578544003.66182, 1578544004.619287]","[United Kingdom, United Kingdom]","[37.220.22.180, 88.150.210.33]",[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"[(53.552, -1.4872), (51.1912, -0.3432)]",亞太電信,http,2,[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"{'48': ['a7U', '4CyGXU', 'Y18n', 'b', 'W:', 'd...",[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"[1578544004.619287, 1578544004.619586, 1578544...",{'20200109': 2},0.0
4,20200106_534_http,"[1578244070.453082, 1578248759.696275, 1578252...","[China, France, Thailand, Thailand, China, Ban...","[119.52.109.64, 83.206.128.205, 146.88.51.234,...",[96:ttUMItUJ3FNnXFFYFF3Ib4su4WnXFIb4sv46Ib4sB4...,"[(42.5272, 125.6753), (48.7142, 7.1898), (13.7...",中華電信,http,36,[96:ttUMItUJ3FNnXFFYFF3Ib4su4WnXFIb4sv46Ib4sB4...,"{'96': ['nXF', 'Y', '3I', 'nXFI', 'I', '4', '4...",[96:TnXFUYU33ImnXFI1I4PUmlXnXNUm4dUm8q3UUKnXFU...,"[1578295954.030631, 1578295954.030895, 1578295...",{'20200106': 36},0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,20200109_317827601065_http_20200109_1632087680...,"[[1578545206.609212, 1578545369.126079, 157854...","[[Russia, Russia, Russia, Russia, Russia, Russ...","[[193.23.3.36, 193.23.3.36, 193.23.3.36, 193.2...",[[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA...,"[[(55.7386, 37.6068), (55.7386, 37.6068), (55....","[台灣碩網, 台灣碩網]","[http, http]","[9, 2]",[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA4...,"{'48': ['a', 'Q5jqXMz', 'A', 'q', ':', '5Rz', ...",[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA4...,"[[1578545206.609212, 1578545206.609449, 157854...","{'20200109': 8, '20200110': 3}",0.0
340,20200107_206158449329_http_20200106_1202590851...,"[[1578333098.110814, 1578398130.044447, 157860...","[[Germany, Russia, United States, Ukraine, Ukr...","[[185.220.101.79, 5.188.210.65, 207.97.174.134...",[[96:Cdo2Hqv5BI93yS1uw1Acz5W5BI93yx5Kx:Cdo2Kv5...,"[[(51.2993, 9.491), (55.7386, 37.6068), (40.71...","[台灣碩網, 台灣碩網, 台灣碩網]","[http, http, http]","[5, 3, 3]",[96:Cdo2Hqv5BI93yS1uw1Acz5W5BI93yx5Kx:Cdo2Kv5B...,"{'96': ['B', 'I', '9', 'yS1uw1', 'B', 'I', '9'...",[96:7+y28P2BI9XyS1uw1sO5Yy28P2BI9Xyn5Kx:7+yX2B...,"[[1578333098.110814, 1578333098.111041, 157833...","{'20200107': 5, '20200110': 3, '20200106': 3}",0.0
341,20200109_85899367448_http_20200109_8589941749_...,"

In [83]:
from datetime import timedelta
min_date = []
def count_date_sessions(timeliolili,ispstroli):
    '''
    GOAL: construct a dict, key=date, value=session#
    Input: list or list of list (element is float)
    Output: dict
    '''
    date_dict = {}
    if type(ispstroli)==list: #巢狀結構
        for isp,timeli in zip(ispstroli,timeliolili):
            for timestamp in timeli:
                dt_object = datetime.fromtimestamp(timestamp)
#                 dt_object = dt_object +timedelta(minutes=10)       
                date_str = str(dt_object).split(" ")[0]
                date_str = date_str.replace('-','')
                if date_str == '20200105': #ad=hot 切邊
                    continue
                    min_date.append(timestamp)
#                     print("!!!0105",timestamp)
#                     date_str = '20200106'
                elif date_str == '20200113':
                    print(timestamp)
#                     date_str = '20200112'
                try:
                    tmp = date_dict[isp+'_'+date_str]
                    tmp = tmp + 1
                    date_dict[isp+'_'+date_str] = tmp
                except KeyError:
                    date_dict[isp+'_'+date_str] = 1      
    else:
        for timestamp in timeliolili:
            dt_object = datetime.fromtimestamp(timestamp)
#             dt_object = dt_object +timedelta(minutes=10)
            date_str = str(dt_object).split(" ")[0]
            date_str = date_str.replace('-','')
            if date_str == '20200105': #ad=hot 切邊
                continue
                min_date.append(timestamp)
#                 print("!!!0105",timestamp)
#                 date_str = '20200106'
            elif date_str == '20200113':
                print(timestamp)
#                 date_str = '20200112'            
            try:
                tmp = date_dict[ispstroli+'_'+date_str]
                tmp = tmp + 1
                date_dict[ispstroli+'_'+date_str] = tmp
            except KeyError:
                date_dict[ispstroli+'_'+date_str] = 1        
    return date_dict        
            
    

combine_dict = {}
for proto in tqdm(protocols_need):
    df = agg_proto_df[proto]
    df['isp_date_sessioncount'] = df.apply(lambda x:count_date_sessions(x.timestamp,x.isp),axis=1)
    temp_li = df['isp_date_sessioncount'].tolist()
    for di in temp_li:
        for k,v in di.items():
            key = proto+"_"+k
            try:
                temp = combine_dict[key]
                temp = temp + v
                combine_dict[key] = temp
            except KeyError:
                combine_dict[key] = v    
#     break
session_count = pd.DataFrame.from_dict(combine_dict,orient='index')
session_count.columns = ['session_num']
session_count

100%|██████████| 11/11 [02:46<00:00, 15.11s/it]


,session_num
http_亞太電信_20200108,40618
http_亞太電信_20200109,84002
http_亞太電信_20200106,92415
http_亞太電信_20200107,91651
http_亞太電信_20200110,67141
...,...
sip_台灣大哥大_20200107,468
sip_台灣大哥大_20200109,23
sip_台灣大哥大_20200110,671
sip_台灣大哥大_20200112,21898


In [69]:
# from datetime import timedelta
# dt_object = datetime.fromtimestamp(min(min_date))
# dt_object = dt_object +timedelta(minutes=10)
# date_str = str(dt_object).split(" ")#[0]
# date_str

# # td = datetime.timedelta(hours=8)
# # now = datetime.datetime.now() + td
# # now = now.strftime('%Y-%m-%d %H%M%S')

# temp1 = merge_df[merge_df.index.str.contains('http')]
# temp1.ratio.mean()

['2020-01-06', '00:00:23.240960']

In [84]:
merge_df = pd.merge(packet_count,session_count,left_index=True,right_index=True)
merge_df['ratio'] = merge_df['packet_num']/merge_df['session_num']
merge_df #可能會<1??




,packet_num,session_num,ratio
http_中嘉寬頻_20200106,126364749,71266,1773.142158
http_中嘉寬頻_20200107,111947097,73481,1523.483581
http_中嘉寬頻_20200108,93768133,64073,1463.457822
http_中嘉寬頻_20200109,91728334,71790,1277.731355
http_中嘉寬頻_20200110,11266917,6342,1776.555818
...,...,...,...
sip_台灣碩網_20200108,988,93,10.623656
sip_台灣碩網_20200109,1983,163,12.165644
sip_台灣碩網_20200110,337372,23696,14.237508
sip_台灣碩網_20200111,69562,4911,14.164529


In [86]:
def count_packets(session_dict,proto):
    packet_num = 0
    for k,v in session_dict.items():
        key = proto+"_"+k
        try:
            packet_num = packet_num + int(v*merge_df.loc[key,'ratio'])
        except KeyError:
            dt = key.split("_")[-1]
            temp1 = merge_df[merge_df.index.str.contains(dt)]
            temp1 = temp1.ratio.mean()
            temp2 = merge_df[merge_df.index.str.contains(proto)]
            temp2 = temp2.ratio.mean()            
            ratio = (temp1+temp2)/2
            packet_num = packet_num + int(v*ratio)
            
    return packet_num
def count_sessions(intoli):
    if type(intoli)==list:
        return sum(intoli)
    else:
        return intoli
def distinct_ipcountry_num(liolili):
    if type(liolili[0]) == list:
        return len(set(functools.reduce(operator.iconcat, liolili, [])))
    else:
        return len(set(liolili))    
for proto in protocols_need:
    df = agg_proto_df[proto]
    df['total_packets_num'] = df['isp_date_sessioncount'].apply(count_packets,args=(proto,))
    df['total_sessions_num'] = df['session_num'].apply(count_sessions)
    df['total_ip_num'] = df['src_ip'].map(distinct_ipcountry_num)
    df['total_country_num'] = df['country'].map(distinct_ipcountry_num)
pickle.dump(obj=agg_proto_df,file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/crossisp_statistics_dfdict.pkl','wb')) #改!!
agg_proto_df['http']

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,signature,ssdeep_clean,clusterkey_timelist,date_count,sum,isp_date_sessioncount,total_packets_num,total_sessions_num,total_ip_num,total_country_num
0,20200108_8589990159_http,"[1578491422.648805, 1578491434.488182, 1578491...","[South Korea, South Korea, South Korea, South ...","[122.41.87.188, 122.41.87.188, 122.41.87.188, ...",[48:o0SP/FgTwfTihvZMlsSp0UUwaIqF6GY18nR:PSPdMw...,"[(37.5112, 126.9741), (37.5112, 126.9741), (37...",亞太電信,http,118,[48:o0SP/FgTwfTihvZMlsSp0UUwaIqF6GY18nR:PSPdMw...,"{'48': ['a', 'q', 'F6GY18n', 'R', 'M', '5Y1k']}",[48:Ga7UCqF6GY18nwiLkcRq09jcRq09uMRRdJtYMY8:hg...,"[1578491500.667118, 1578491500.667435, 1578491...","{'20200108': 7, '20200109': 111}",0.0,"{'亞太電信_20200108': 7, '亞太電信_20200109': 111}",21036,118,3,3
1,20200106_8589962872_http,"[1578244875.871224, 1578279973.587717, 1578291...","[Netherlands, Netherlands, Netherlands, Nether...","[51.15.71.134, 51.15.71.134, 51.15.71.134, 51....",[24:LepsAkMVAab5CYD0sVUYaP+shmGvQEjyOFEoW:Leps...,"[(52.3759, 4.8975), (52.3759, 4.8975), (52.375...",亞太電信,http,25,[24:LepsAkMVAab5CYD0sVUYaP+shmGvQEjyOFEoW:Leps...,{'12': ['Bhzeps+hkMVk3HXq6xgzhrHCYPEjyOFE4nBQo...,[12:Bhzeps+hkMVk3HXq6xgzhrHCYPEjyOFE4nBQom:Lep...,"[1578315077.277642, 1578315077.277914, 1578315...","{'20200106': 5, '20200107': 5, '20200108': 2, ...",0.0,"{'亞太電信_20200106': 5, '亞太電信_20200107': 5, '亞太電信...",5434,25,1,1
2,20200106_206158498684_http,"[1578325476.581104, 1578325480.685079]","[Canada, Canada]","[209.99.129.253, 209.99.129.253]",[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,"[(43.6655, -79.4204), (43.6655, -79.4204)]",亞太電信,http,2,[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,{'96': ['rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41u...,[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,"[1578325476.581104, 1578325476.581368, 1578325...",{'20200106': 2},0.0,{'亞太電信_20200106': 2},441,2,1,1
3,20200109_128849077624_http,"[1578544003.66182, 1578544004.619287]","[United Kingdom, United Kingdom]","[37.220.22.180, 88.150.210.33]",[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"[(53.552, -1.4872), (51.1912, -0.3432)]",亞太電信,http,2,[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"{'48': ['a7U', '4CyGXU', 'Y18n', 'b', 'W:', 'd...",[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"[1578544004.619287, 1578544004.619586, 1578544...",{'20200109': 2},0.0,{'亞太電信_20200109': 2},346,2,2,1
4,20200106_534_http,"[1578244070.453082, 1578248759.696275, 1578252...","[China, France, Thailand, Thailand, China, Ban...","[119.52.109.64, 83.206.128.205, 146.88.51.234,...",[96:ttUMItUJ3FNnXFFYFF3Ib4su4WnXFIb4sv46Ib4sB4...,"[(42.5272, 125.6753), (48.7142, 7.1898), (13.7...",中華電信,http,36,[96:ttUMItUJ3FNnXFFYFF3Ib4su4WnXFIb4sv46Ib4sB4...,"{'96': ['nXF', 'Y', '3I', 'nXFI', 'I', '4', '4...",[96:TnXFUYU33ImnXFI1I4PUmlXnXNUm4dUm8q3UUKnXFU...,"[1578295954.030631, 1578295954.030895, 1578295...",{'20200106': 36},0.0,{'中華電信_20200106': 36},31210,36,12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,20200109_317827601065_http_20200109_1632087680...,"[[1578545206.609212, 1578545369.126079, 157854...","[[Russia, Russia, Russia, Russia, Russia, Russ...","[[193.23.3.36, 193.23.3.36, 193.23.3.36, 193.2...",[[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA...,"[[(55.7386, 37.6068), (55.7386, 37.6068), (55....","[台灣碩網, 台灣碩網]","[http, http]","[9, 2]",[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA4...,"{'48': ['a', 'Q5jqXMz', 'A', 'q', ':', '5Rz', ...",[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA4...,"[[1578545206.609212, 1578545206.609449, 157854...","{'20200109': 8, '20200110': 3}",0.0,"{'台灣碩網_20200109': 8, '台灣碩網_20200110': 3}",15332,11,2,2
340,20200107_206158449329_http_20200106_1202590851...,"[[1578333098.110814, 1578398130.044447, 157860...","[[Germany, Russia, United States, Ukraine, Ukr...","[[185.220.101.79, 5.188.210.65, 207.97.174.134...",[[96:Cdo2Hqv5BI93yS1uw1Acz5W5BI93yx5Kx:Cdo2Kv5...,"[[(51.2993, 9.491), (55.7386,

In [ ]:
TODO:
#去讀出所有的IP，然後依照index位置判斷包含哪些timestamp(lioli)，還原他的日期(5號的就丟掉)，搭配對應的isp組成key(可能多個)
#，依照timestamp數量乘上ratio可得packets。

In [79]:
# agg_proto_df['sip'].session_num.apply(sum)

# temp_li = df['isp_date_sessioncount'].tolist()
# combine_dict = {}
# for di in temp_li:
#     for k,v in di.items():
#         key = proto+"_"+k
#         try:
#             temp = combine_dict[key]
#             temp = temp + v
#             combine_dict[key] = temp
#         except KeyError:
#             combine_dict[key] = v
# combine_dict

TypeError: 'int' object is not iterable

## crossisp_crossdates_AS

In [7]:
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"]  #改!!
protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']
file_name = "_".join(sorted(date_li)) #date_li 改!!
min_date = str(min(date_li))
max_date = str(max(date_li))
agg_proto_df = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/crossisp_statistics_dfdict.pkl','rb'))

In [8]:
for i,proto in enumerate(protocols_need):
    if i ==0:
        all_df = agg_proto_df[proto]
    else:
        temp = agg_proto_df[proto]
        all_df = all_df.append(temp)
all_df = all_df.reset_index(drop=True)   
all_df

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,signature,ssdeep_clean,clusterkey_timelist,date_count,sum,isp_date_sessioncount,total_packets_num,total_sessions_num,total_ip_num,total_country_num
0,20200108_8589990159_http,"[1578491422.648805, 1578491434.488182, 1578491...","[South Korea, South Korea, South Korea, South ...","[122.41.87.188, 122.41.87.188, 122.41.87.188, ...",[48:o0SP/FgTwfTihvZMlsSp0UUwaIqF6GY18nR:PSPdMw...,"[(37.5112, 126.9741), (37.5112, 126.9741), (37...",亞太電信,http,118,[48:o0SP/FgTwfTihvZMlsSp0UUwaIqF6GY18nR:PSPdMw...,"{'48': ['a', 'q', 'F6GY18n', 'R', 'M', '5Y1k']}",[48:Ga7UCqF6GY18nwiLkcRq09jcRq09uMRRdJtYMY8:hg...,"[1578491500.667118, 1578491500.667435, 1578491...","{'20200108': 7, '20200109': 111}",0.0,"{'亞太電信_20200108': 7, '亞太電信_20200109': 111}",21036,118,3,3
1,20200106_8589962872_http,"[1578244875.871224, 1578279973.587717, 1578291...","[Netherlands, Netherlands, Netherlands, Nether...","[51.15.71.134, 51.15.71.134, 51.15.71.134, 51....",[24:LepsAkMVAab5CYD0sVUYaP+shmGvQEjyOFEoW:Leps...,"[(52.3759, 4.8975), (52.3759, 4.8975), (52.375...",亞太電信,http,25,[24:LepsAkMVAab5CYD0sVUYaP+shmGvQEjyOFEoW:Leps...,{'12': ['Bhzeps+hkMVk3HXq6xgzhrHCYPEjyOFE4nBQo...,[12:Bhzeps+hkMVk3HXq6xgzhrHCYPEjyOFE4nBQom:Lep...,"[1578315077.277642, 1578315077.277914, 1578315...","{'20200106': 5, '20200107': 5, '20200108': 2, ...",0.0,"{'亞太電信_20200106': 5, '亞太電信_20200107': 5, '亞太電信...",5434,25,1,1
2,20200106_206158498684_http,"[1578325476.581104, 1578325480.685079]","[Canada, Canada]","[209.99.129.253, 209.99.129.253]",[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,"[(43.6655, -79.4204), (43.6655, -79.4204)]",亞太電信,http,2,[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,{'96': ['rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41u...,[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,"[1578325476.581104, 1578325476.581368, 1578325...",{'20200106': 2},0.0,{'亞太電信_20200106': 2},441,2,1,1
3,20200109_128849077624_http,"[1578544003.66182, 1578544004.619287]","[United Kingdom, United Kingdom]","[37.220.22.180, 88.150.210.33]",[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"[(53.552, -1.4872), (51.1912, -0.3432)]",亞太電信,http,2,[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"{'48': ['a7U', '4CyGXU', 'Y18n', 'b', 'W:', 'd...",[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"[1578544004.619287, 1578544004.619586, 1578544...",{'20200109': 2},0.0,{'亞太電信_20200109': 2},346,2,2,1
4,20200106_534_http,"[1578244070.453082, 1578248759.696275, 1578252...","[China, France, Thailand, Thailand, China, Ban...","[119.52.109.64, 83.206.128.205, 146.88.51.234,...",[96:ttUMItUJ3FNnXFFYFF3Ib4su4WnXFIb4sv46Ib4sB4...,"[(42.5272, 125.6753), (48.7142, 7.1898), (13.7...",中華電信,http,36,[96:ttUMItUJ3FNnXFFYFF3Ib4su4WnXFIb4sv46Ib4sB4...,"{'96': ['nXF', 'Y', '3I', 'nXFI', 'I', '4', '4...",[96:TnXFUYU33ImnXFI1I4PUmlXnXNUm4dUm8q3UUKnXFU...,"[1578295954.030631, 1578295954.030895, 1578295...",{'20200106': 36},0.0,{'中華電信_20200106': 36},31210,36,12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,20200106_14243_sip_20200106_465_sip_20200106_8...,"[[1578239440.238951, 1578239443.45521, 1578239...","[[United States, United States, United States,...","[[104.140.188.2, 104.206.128.46, 104.206.128.2...",[[12:qYlim7mDLRfAaAz85g9qX6oa+sCssVWGY07GiDv+5...,"[[(37.751, -97.822), (37.751, -97.822), (37.75...","[亞太電信, 中華電信, 台灣基礎開發, 台灣之星, 台灣固網, 台灣碩網, 凱擘, 台固媒...","[sip, sip, sip, sip, sip, sip, sip, sip, sip, ...","[1146, 3476, 688, 1030, 1917, 605, 855, 454, 1...",[12:qYlim7mDLRfAaAz85g9qX6oa+sCssVWGY07GiDv+5g...,"{'12': ['A', 'z', '5g9qX6oa+sCssVWGY07GiDv+5gT...",[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...,"[[1578293229.060841, 1578293229.061113, 157829...","{'20200105': 46, '20200106': 2275, '20200107':...",NaN,"{'亞太電信_20200106': 277, '亞太電信_20200107': 201, '...",144740,11930,75,5
1990,20200106_77309423428_sip_20200106_13013_sip_20...,"[[1578320613.049779, 1578320613.105093, 157832...","[[Estonia, Estonia, Estonia, Es

In [88]:
#find all ips
def src_ip_flat(liolili):
    if type(liolili[0])==list:
        return list(set(functools.reduce(operator.iconcat, liolili, [])))
    else:
        return list(set(liolili))
all_df['unique_ips'] = all_df['src_ip'].map(src_ip_flat)
all_ips = all_df.unique_ips.tolist()
all_ips = list(set(functools.reduce(operator.iconcat, all_ips, [])))#sum(all_ips,[])
all_ips = sorted(all_ips)
# col_li = ['Country']
# col_li.extend(all_df.idx.tolist())
col_li = all_df.idx.tolist()
# col_li = [x for x in col_li if 'tds' not in x]
# col_li.append('tds') #改!!
jc_matrix = pd.DataFrame(0, index=all_ips, columns=col_li)

In [90]:
ips

[['66.249.65.188',
  '66.249.79.137',
  '66.249.71.43',
  '66.249.73.76',
  '66.249.79.205',
  '66.249.71.25',
  '66.249.65.238',
  '66.249.71.86',
  '66.249.79.123',
  '66.249.71.158',
  '66.249.79.158',
  '66.249.69.46',
  '66.249.65.64',
  '66.249.71.83',
  '66.249.70.19',
  '66.249.71.118',
  '66.249.79.97',
  '66.249.65.181',
  '66.249.71.124',
  '66.249.71.90',
  '66.249.66.201',
  '66.249.79.196',
  '66.249.64.239',
  '66.249.69.43',
  '66.249.79.27',
  '66.249.79.142',
  '66.249.71.43',
  '66.249.73.86',
  '66.249.79.203',
  '66.249.79.185',
  '66.249.65.187',
  '66.249.71.154',
  '66.249.79.28',
  '66.249.71.92',
  '66.249.69.166',
  '66.249.79.83',
  '66.249.65.188',
  '66.249.71.91',
  '66.249.66.21',
  '66.249.69.30',
  '66.249.66.5',
  '66.249.64.64',
  '66.249.64.52',
  '66.249.65.188',
  '66.249.75.126',
  '66.249.79.183',
  '66.249.64.95',
  '66.249.64.148',
  '66.249.71.92',
  '66.249.71.8',
  '66.249.64.231',
  '66.249.75.72',
  '66.249.71.113',
  '66.249.68.6',
  '66

In [91]:
for col in tqdm(jc_matrix.columns.tolist()):
    if 'tds' in col: # 統一tds欄位，如果有的話
        select_df = all_df[all_df.idx.str.contains('tds')]
        ip_li = list(select_df.unique_ips.values)
        try:
            for ips in ip_li:
                jc_matrix.loc[ips,'tds'] = 1
#             jc_matrix = jc_matrix.drop([col])
        except IndexError:
            print("Didn't load tds protocol to all_ip. SKIPPING!")
            pass
    else:
        select_df = all_df[all_df.idx == col]
        ip_li = list(select_df.unique_ips.values)
        for ips in ip_li:      
            jc_matrix.loc[ips,col] = 1
# jc_matrix['np_array'] = list(jc_matrix.values)#.ravel()
jc_matrix

100%|██████████| 1994/1994 [00:02<00:00, 847.87it/s] 


,20200108_8589990159_http,20200106_8589962872_http,20200106_206158498684_http,20200109_128849077624_http,20200106_534_http,20200109_8589950399_http,20200109_51539620339_http,20200111_489626283577_http,20200106_8589966256_http,20200106_25769835233_http,...,20200106_42949704991_sip,20200106_17559_sip,20200106_17467_sip,20200109_2562_sip,20200106_188978602930_sip,20200106_14243_sip_20200106_465_sip_20200106_8452_sip_20200106_8589960531_sip_20200106_1876_sip_20200106_171798691856_sip_20200106_3929_sip_20200106_26461_sip_20200106_1589_sip_20200106_17559_sip,20200106_77309423428_sip_20200106_13013_sip_20200106_51539621821_sip_20200109_43753_sip_20200111_266287981719_sip_20200106_26978_sip_20200110_24448_sip_20200108_94489305705_sip_20200106_10024_sip_20200108_3875_sip_20200109_4186_sip_20200108_25769829215_sip_20200106_8589944070_sip_20200106_17179898116_sip_20200110_688_sip_20200106_13657_sip_20200106_42949704991_sip,20200106_8589948585_sip_20200106_15702_sip_20200106_17467_sip,20200110_34359759799_sip_20200109_32722_sip_20200106_26617_sip_20200109_2562_sip,20200111_1065151919515_sip_20200106_541165880611_sip_20200111_154618826274_sip_20200106_188978602930_sip
1.0.128.212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.0.131.182,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.0.131.207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.0.131.40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.0.133.176,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.98.147.203,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99.98.99.67,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99.99.139.67,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99.99.194.84,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
jc_matrix_stat = jc_matrix.append(pd.Series(jc_matrix.sum(),name='stat'))
jc_matrix_stat['np_array'] = list(jc_matrix_stat.values)
def sum_arr(npy):
    return sum(npy)
jc_matrix_stat['sum'] = jc_matrix_stat.np_array.apply(sum_arr)
jc_matrix_stat

,20200108_8589990159_http,20200106_8589962872_http,20200106_206158498684_http,20200109_128849077624_http,20200106_534_http,20200109_8589950399_http,20200109_51539620339_http,20200111_489626283577_http,20200106_8589966256_http,20200106_25769835233_http,...,20200106_17467_sip,20200109_2562_sip,20200106_188978602930_sip,20200106_14243_sip_20200106_465_sip_20200106_8452_sip_20200106_8589960531_sip_20200106_1876_sip_20200106_171798691856_sip_20200106_3929_sip_20200106_26461_sip_20200106_1589_sip_20200106_17559_sip,20200106_77309423428_sip_20200106_13013_sip_20200106_51539621821_sip_20200109_43753_sip_20200111_266287981719_sip_20200106_26978_sip_20200110_24448_sip_20200108_94489305705_sip_20200106_10024_sip_20200108_3875_sip_20200109_4186_sip_20200108_25769829215_sip_20200106_8589944070_sip_20200106_17179898116_sip_20200110_688_sip_20200106_13657_sip_20200106_42949704991_sip,20200106_8589948585_sip_20200106_15702_sip_20200106_17467_sip,20200110_34359759799_sip_20200109_32722_sip_20200106_26617_sip_20200109_2562_sip,20200111_1065151919515_sip_20200106_541165880611_sip_20200111_154618826274_sip_20200106_188978602930_sip,np_array,sum
1.0.128.212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1.0.131.182,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1.0.131.207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1.0.131.40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3
1.0.133.176,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.98.99.67,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
99.99.139.67,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7
99.99.194.84,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
99.99.201.242,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [100]:
jc_matrix_stat = jc_matrix_stat.drop(['np_array'],axis=1)
pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/'
pickle.dump(file=open(pickle_dir+'crossispdates_clusters_ips_stat_df_'+str(min_date)+'_'+str(max_date)+'.pkl','wb'),
            obj=jc_matrix_stat,protocol=pickle.HIGHEST_PROTOCOL)
print('One hot統計df:',pickle_dir+'crossispdates_clusters_ips_stat_df_'+str(min_date)+'_'+str(max_date)+'.pkl')

One hot統計df: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/crossispdates_clusters_ips_stat_df_20200106_20200112.pkl


In [128]:
jc_matrix_new = jc_matrix.append(pd.Series(jc_matrix.sum(),name='stat'))
bad_col = []
for col in jc_matrix_new.columns:
    try:
        if jc_matrix_new.loc['stat',col] > 1:
            continue
    except ValueError:
        bad_col.append(col)
        print(col)
bad_col = list(set(bad_col))        
for bc in bad_col:
    temp = jc_matrix_new[bc].iloc[:,0]
    jc_matrix_new = jc_matrix_new.drop([bc],axis=1)
    jc_matrix_new = pd.concat((jc_matrix_new,temp),axis=1)
#         print(col)
assert len(jc_matrix_new.columns)==len(set(jc_matrix_new.columns))

20200106_13_ssh
20200111_1_ssh
20200111_1_ssh
20200106_13_ssh


In [129]:

col_need = []
for col in jc_matrix_new.columns:
    if col == 'np_array':
        continue
    if jc_matrix_new.loc['stat',col] > 1:
        col_need.append(col)
jc_matrix_new = jc_matrix_new[col_need]
#濾除row:
jc_matrix_new['np_array'] = list(jc_matrix_new.values)#.ravel()
jc_matrix_new['sum'] = jc_matrix_new.np_array.apply(sum_arr)
#要存下統計變量!刪掉np_array
jc_matrix_new = jc_matrix_new[jc_matrix_new['sum']>1]
jc_matrix_new

,20200108_8589990159_http,20200109_128849077624_http,20200106_534_http,20200106_8589966256_http,20200106_25769835233_http,20200106_5305_http,20200106_14092_http,20200107_22065_http,20200110_4673_http,20200106_85899377344_http,...,20200109_2562_sip,20200106_188978602930_sip,20200106_14243_sip_20200106_465_sip_20200106_8452_sip_20200106_8589960531_sip_20200106_1876_sip_20200106_171798691856_sip_20200106_3929_sip_20200106_26461_sip_20200106_1589_sip_20200106_17559_sip,20200106_77309423428_sip_20200106_13013_sip_20200106_51539621821_sip_20200109_43753_sip_20200111_266287981719_sip_20200106_26978_sip_20200110_24448_sip_20200108_94489305705_sip_20200106_10024_sip_20200108_3875_sip_20200109_4186_sip_20200108_25769829215_sip_20200106_8589944070_sip_20200106_17179898116_sip_20200110_688_sip_20200106_13657_sip_20200106_42949704991_sip,20200110_34359759799_sip_20200109_32722_sip_20200106_26617_sip_20200109_2562_sip,20200111_1065151919515_sip_20200106_541165880611_sip_20200111_154618826274_sip_20200106_188978602930_sip,20200106_13_ssh,20200111_1_ssh,np_array,sum
1.0.131.40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3
1.0.134.96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4
1.0.151.133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
1.0.153.160,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
1.0.153.224,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.89.92.55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",17
99.93.68.47,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4
99.95.239.15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
99.99.139.67,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7


In [130]:
jc_matrix = jc_matrix_new.iloc[0:len(jc_matrix_new)-1]
jc_matrix = jc_matrix.sort_values(['sum'],ascending=False)
jc_matrix = jc_matrix.drop(['sum','np_array'],axis=1)
jc_matrix['np_array'] = list(jc_matrix.values)
jc_matrix

,20200108_8589990159_http,20200109_128849077624_http,20200106_534_http,20200106_8589966256_http,20200106_25769835233_http,20200106_5305_http,20200106_14092_http,20200107_22065_http,20200110_4673_http,20200106_85899377344_http,...,20200106_17559_sip,20200109_2562_sip,20200106_188978602930_sip,20200106_14243_sip_20200106_465_sip_20200106_8452_sip_20200106_8589960531_sip_20200106_1876_sip_20200106_171798691856_sip_20200106_3929_sip_20200106_26461_sip_20200106_1589_sip_20200106_17559_sip,20200106_77309423428_sip_20200106_13013_sip_20200106_51539621821_sip_20200109_43753_sip_20200111_266287981719_sip_20200106_26978_sip_20200110_24448_sip_20200108_94489305705_sip_20200106_10024_sip_20200108_3875_sip_20200109_4186_sip_20200108_25769829215_sip_20200106_8589944070_sip_20200106_17179898116_sip_20200110_688_sip_20200106_13657_sip_20200106_42949704991_sip,20200110_34359759799_sip_20200109_32722_sip_20200106_26617_sip_20200109_2562_sip,20200111_1065151919515_sip_20200106_541165880611_sip_20200111_154618826274_sip_20200106_188978602930_sip,20200106_13_ssh,20200111_1_ssh,np_array
5.188.86.168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5.182.39.94,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5.188.86.167,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5.188.87.49,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5.188.86.206,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37.213.109.136,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
37.213.113.79,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
187.87.151.146,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
187.87.13.161,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
#檢查!! 不能有assertion err!!!
def sum_val(npy):
    return sum(npy)
jc_matrix['sum'] = jc_matrix['np_array'].apply(sum_val)
temp = jc_matrix[jc_matrix['sum'] == 0]
weird_ips = temp.index.tolist()
assert len(weird_ips) == 0
jc_matrix = jc_matrix.drop(['sum'],axis=1)
gc.collect()

In [ ]:
jc_matrix_stat = jc_matrix.append(pd.Series(jc_matrix.sum(),name='stat'))
# jc_matrix_stat['np_array'] = list(jc_matrix_stat.values)
def sum_arr(npy):
    return sum(npy)
jc_matrix_stat['sum'] = jc_matrix_stat.np_array.apply(sum_arr)
jc_matrix_stat = jc_matrix_stat.drop(['np_array'],axis=1)
# pickle_dir = '/home/antslab/spark_data/pcap_inter/2020_01_06/中華電信/case_pickles/'
pickle.dump(file=open(pickle_dir+'crossispdates_clusters_ips_stat_afterFilter_df_'+str(min_date)+'_'+str(max_date)+'.pkl','wb')
            ,obj=jc_matrix_stat,protocol=pickle.HIGHEST_PROTOCOL)
print("把col=1,row=1以下的濾掉之統計df:",pickle_dir+'crossispdates_clusters_ips_stat_afterFilter_df_'+str(min_date)+'_'+str(max_date)+'.pkl')
jc_matrix_stat

In [ ]:
jc_matrix_stat[jc_matrix_stat['sum']<2] #要0 rows

In [ ]:
def calc_jac(c_value,t_value):
    '''
    GOAL: 同時考量jaccrd計算方式，與人類直覺計算方式
    '''
    j_s = jaccard_score(c_value, t_value)
    c_s = cosine_similarity([c_value], [t_value])[0][0]
    one_portion = max(sum(c_value),sum(t_value))/len(t_value) #最大長度的1的數量
    final_score = (c_s*one_portion)+(j_s*(1-one_portion))
    return final_score
#     return jaccard_score(c_value, t_value)

def calc_cos(c_value,t_value):
    return cosine_similarity(c_value, t_value)


In [ ]:
gc.collect()
thr_li= [0.1,0.5,0.9] #,0.8,0.7,0.6,0.5,0.4,0.3,0.2
# thr_li = [0.1,0.5]
for thr in tqdm(thr_li):
    jc_dict = {}
    ip_li = jc_matrix.index.tolist() #pandas
#     ip_li = list(jc_matrix.index.to_numpy()) #koalas
    used_ip = []
    for ip in ip_li:
        if ip in used_ip: #合併過得拿掉 single label
            continue
        t_value = jc_matrix.loc[ip,'np_array']# pandas
#         t_value = jc_matrix.loc[ip].values #koalas
        jc_calc = jc_matrix[~jc_matrix.index.isin(used_ip)] #合併過得拿掉 single label
        jc_calc = jc_calc[jc_calc.index != ip] #自己的不比 singleLabel
    #     jc_calc = jc_matrix[jc_matrix.index != ip] #自己的不比    multilabel
        jc_calc['jc_score'] = jc_calc.np_array.apply(calc_jac,args=(t_value,)) #得到t跟每個c的分數
    #     jc_calc['co_score'] = jc_calc.np_array.apply(calc_cos,args=(t_value,)) #得到t跟每個c的分數
        combine_df = jc_calc[jc_calc['jc_score']>thr] #所設定的相似度分數
        c_ips_li = combine_df.index.tolist() #跟這個IP具高度相似度的IPs
        if len(c_ips_li)>0:
            jc_dict[ip] = c_ips_li
            used_ip.extend(c_ips_li) #合併過的不要再比
            used_ip.append(ip) #用過的不要再比

    loner_ip = list(set(ip_li) - set(used_ip))
    min_date = str(min(date_li))
    pickle_path = pickle_dir+'crossispdates_CorrelateIP_APTIP_thr'+str(thr)+'_'+file_name+'.pkl' #改!!
    pickle.dump(obj=(jc_dict,loner_ip),file=open(pickle_path,'wb'),protocol=pickle.HIGHEST_PROTOCOL)
    print("threshold =",thr,"(jaccard_dictionary,loner_ip) save path:",pickle_path)
    print('集團數量(IP>1,score>'+str(thr)+'):',len(jc_dict),"LonerIP數量:",len(loner_ip))

In [ ]:
1

# TEST code

In [21]:
all_df.total_sessions_num.sum()
# all_df.total_packets_num.sum()

77687763

In [17]:
# count total country num
def count_country(liorlili):
    if type(liorlili[0])==list:
        return list(set(functools.reduce(operator.iconcat, liorlili, [])))
    else:
        return list(set(liorlili))
all_df['srcip_set'] = all_df.src_ip.map(count_country)
temp = all_df.srcip_set.tolist()
temp = list(set(functools.reduce(operator.iconcat, temp, [])))
len(temp)

397324

In [24]:
# df_http = agg_proto_df['http']
# df_http['srcip_set'] = df_http.src_ip.map(count_country)
# temp = df_http.srcip_set.tolist()
# temp = list(set(functools.reduce(operator.iconcat, temp, [])))
# len(temp)

df_http = agg_proto_df['http']
df_http['country_set'] = df_http.country.map(count_country)
temp = df_http.country_set.tolist()
temp = list(set(functools.reduce(operator.iconcat, temp, [])))
len(temp)

198

In [23]:
# df_http.total_sessions_num.sum()
df_http.total_packets_num.sum()

7172206284

In [25]:
len(df_http)

344

In [49]:
tt = df_http.sort_values('total_sessions_num',ascending=False).head(31)
from collections import Counter
def countryip_count(liolili):
    if type(liolili[0]) == list:
        temp =  list(functools.reduce(operator.iconcat, liolili, []))
        x= dict(Counter(temp))  
        return dict(sorted(x.items(), key=lambda item: item[1],reverse=True))
                   
    else:
        x = dict(Counter(liolili))  
        return dict(sorted(x.items(), key=lambda item: item[1],reverse=True))
tt['count_country'] = tt['country'].map(countryip_count)
tt['count_ip']  = tt['src_ip'].map(countryip_count)
tt

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,...,sum,isp_date_sessioncount,total_packets_num,total_sessions_num,total_ip_num,total_country_num,srcip_set,country_set,count_country,count_ip
252,20200106_13656_http_20200106_2984_http_2020010...,"[[1578239437.622562, 1578239439.00718, 1578239...","[[United States, United States, United States,...","[[66.249.65.188, 66.249.79.137, 66.249.71.43, ...",[[48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691u...,"[[(37.751, -97.822), (37.751, -97.822), (37.75...","[亞太電信, 亞太電信, 亞太電信, 亞太電信, 亞太電信, 亞太電信, 中華電信, 中華電...","[http, http, http, http, http, http, http, htt...","[183041, 64609, 18631, 1062, 62, 2, 326422, 86...",[48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691uw...,...,0.0,"{'亞太電信_20200106': 57038, '亞太電信_20200107': 3356...",2900868695,2735459,44486,178,"[40.77.167.47, 182.16.162.66, 45.169.219.180, ...","[Switzerland, Croatia, Saudi Arabia, Burundi, ...","{'United States': 2455292, 'Russia': 58959, 'C...","{'156.96.155.243': 20272, '52.228.41.177': 112..."
253,20200106_295_http_20200107_35988_http_20200106...,"[[1578239440.513196, 1578239443.592105, 157823...","[[Iran, Russia, United States, United States, ...","[[79.132.201.185, 212.41.14.170, 216.244.66.24...",[[12:rHXq6xmMVkExShI/XES+AoGY1uqh5U9Z1J3vhm:Da...,"[[(35.698, 51.4115), (56.003, 92.7993), (47.72...","[亞太電信, 亞太電信, 亞太電信, 中華電信, 中華電信, 中華電信, 中華電信, 中華電...","[http, http, http, http, http, http, http, htt...","[81808, 2169, 80, 268588, 130847, 9758, 15, 7,...",[12:rHXq6xmMVkExShI/XES+AoGY1uqh5U9Z1J3vhm:Da7...,...,0.0,"{'亞太電信_20200106': 9327, '亞太電信_20200107': 9635,...",1041512399,1188705,94601,189,"[85.204.221.120, 79.132.19.197, 190.6.199.22, ...","[Senegal, Switzerland, Croatia, Burundi, Saudi...","{'United States': 345166, 'China': 326451, 'Ru...","{'5.101.0.209': 25014, '216.244.66.201': 17363..."
254,20200106_51539658953_http_20200106_13915_http_...,"[[1578243129.379452, 1578243147.033231, 157824...","[[China, China, China, China, China, China, Ho...","[[103.140.126.152, 103.140.126.152, 103.140.12...",[[12:72wXcHnMQ7s95HfSBBBBBBBSjmHXq/Ish+j51h1Jc...,"[[(34.7725, 113.7266), (34.7725, 113.7266), (3...","[亞太電信, 亞太電信, 亞太電信, 亞太電信, 中華電信, 中華電信, 中華電信, 台灣基...","[http, http, http, http, http, http, http, htt...","[48240, 882, 66, 11, 1788, 1345, 158, 67071, 1...",[12:72wXcHnMQ7s95HfSBBBBBBBSjmHXq/Ish+j51h1JcZ...,...,0.0,"{'亞太電信_20200106': 384, '亞太電信_20200107': 3196, ...",542600338,541788,36956,177,"[81.23.203.143, 190.6.199.22, 99.170.44.176, 1...","[Switzerland, Croatia, Burundi, Saudi Arabia, ...","{'China': 307868, 'Hong Kong': 38503, 'United ...","{'43.241.252.98': 4858, '202.79.173.12': 4437,..."
299,20200106_8309_http_20200106_5367_http_20200106...,"[[1578244516.892598, 1578244624.073799, 157824...","[[United States, United States, United States,...","[[216.244.66.199, 216.244.66.236, 216.244.66.1...",[[24:tAB6fG+fG+XEQa7MV9IkEc1wgxIGYKho59:+B6O+O...,"[[(47.7229, -122.1961), (47.7229, -122.1961), ...","[中華電信, 台灣基礎開發, 台灣固網, 台灣碩網, 台灣碩網, 台灣碩網, 凱擘, 台固媒...","[http, http, http, http, http, http, http, htt...","[1488, 297, 4051, 98251, 627, 3, 15702, 3356, ...",[24:tAB6fG+fG+XEQa7MV9IkEc1wgxIGYKho59:+B6O+O+...,...,0.0,"{'中華電信_20200106': 129, '中華電信_20200107': 112, '...",417231572,445302,34040,178,"[64.53.143.72, 182.16.162.66, 78.165.109.166, ...","[Senegal, Switzerland, Croatia, Saudi Arabia, ...","{'United States': 254870, 'China': 82901, 'Hon...","{'216.244.66.249': 24953, '216.244.66.240': 12..."
261,20200106_19379_http_20200106_10410_http_202001...,"[[1578240398.599221, 1578241052.004457, 157824...","[[Japan, Japan, Japan, Japan, Japan, Japan, Ja...","[[139.162.108.129, 139.162.79.87, 139.162.79.8...",[[12:BSwoyR9Aj3HXq6xYYDwZNkZrcQJ1Zu1Is0+R5a8m:...,"[[(35.6882, 139.7532), (35.6882, 139.7532), (3...","[亞太電信, 亞太電信, 亞太電信, 中華電信, 中華電信, 台灣基礎開發, 台灣基礎開發,...","[http, http, http, http, http, http, http, htt...","[2709, 2157, 1408, 6993, 736, 23581, 4734, 930...",[12:BSwoyR9Aj3HXq6xYYDwZNkZrcQJ1Zu1Is0+R5a8m:+...,...,

In [88]:
tt['total_sessions_num'].tolist()

[2735459,
 1188705,
 541788,
 445302,
 428167,
 328367,
 264115,
 195883,
 150594,
 128740,
 111201,
 107047,
 78266,
 62876,
 52158,
 50189,
 44590,
 35071,
 34045,
 30351,
 28427,
 26169,
 24477,
 17475,
 14496,
 14168,
 10023,
 8754,
 7264,
 6912,
 6457]

In [33]:
tt.loc[331,'idx']

'20200106_8589935402_http_20200106_4685_http_20200106_2971_http_20200106_4607_http'

In [57]:
idx_li = tt.idx.tolist()
for idx in idx_li:
    if '_550_' in idx:
        print(idx)

20200106_152_http_20200106_8589947030_http_20200106_5915_http_20200106_6145_http_20200106_6314_http_20200106_17876_http_20200107_343597459214_http_20200106_467_http_20200106_550_http_20200106_4795_http_20200106_282_http_20200106_20066_http_20200106_27213_http_20200106_8589966989_http_20200106_283_http_20200106_2747_http_20200106_15058_http_20200106_59_http_20200106_8589963178_http_20200106_2177_http_20200108_1784_http_20200106_1105_http_20200106_4014_http_20200106_34367_http_20200106_8589935559_http_20200106_233_http_20200106_12_http_20200106_1706_http_20200106_17179873818_http_20200106_60129545782_http_20200106_3492_http_20200106_137438957183_http_20200106_3022_http_20200106_3039_http_20200106_8589937662_http_20200106_4636_http_20200106_17179889191_http_20200106_60129546916_http_20200107_266288003594_http_20200110_584115588973_http_20200106_1946_http_20200106_5423_http_20200106_22919_http_20200106_8589956365_http_20200106_128849024593_http_20200106_18_http_20200106_32_http_20200106_84

In [90]:
# kk = '20200106_152_http_20200106_8589947030_http_20200106_5915_http_20200106_6145_http_20200106_6314_http_20200106_17876_http_20200107_343597459214_http_20200106_467_http_20200106_550_http_20200106_4795_http_20200106_282_http_20200106_20066_http_20200106_27213_http_20200106_8589966989_http_20200106_283_http_20200106_2747_http_20200106_15058_http_20200106_59_http_20200106_8589963178_http_20200106_2177_http_20200108_1784_http_20200106_1105_http_20200106_4014_http_20200106_34367_http_20200106_8589935559_http_20200106_233_http_20200106_12_http_20200106_1706_http_20200106_17179873818_http_20200106_60129545782_http_20200106_3492_http_20200106_137438957183_http_20200106_3022_http_20200106_3039_http_20200106_8589937662_http_20200106_4636_http_20200106_17179889191_http_20200106_60129546916_http_20200107_266288003594_http_20200110_584115588973_http_20200106_1946_http_20200106_5423_http_20200106_22919_http_20200106_8589956365_http_20200106_128849024593_http_20200106_18_http_20200106_32_http_20200106_843_http_20200106_8589945239_http_20200106_357_http_20200106_1587_http_20200106_24009_http_20200106_1708_http_20200106_8589936509_http_20200106_17179881583_http_20200106_2761_http_20200106_35053_http_20200106_3248_http_20200106_51539626497_http_20200106_51539607673_http'.split('_http_')
# len(kk)

60

In [58]:
take_AP = tt[tt.idx=='20200106_152_http_20200106_8589947030_http_20200106_5915_http_20200106_6145_http_20200106_6314_http_20200106_17876_http_20200107_343597459214_http_20200106_467_http_20200106_550_http_20200106_4795_http_20200106_282_http_20200106_20066_http_20200106_27213_http_20200106_8589966989_http_20200106_283_http_20200106_2747_http_20200106_15058_http_20200106_59_http_20200106_8589963178_http_20200106_2177_http_20200108_1784_http_20200106_1105_http_20200106_4014_http_20200106_34367_http_20200106_8589935559_http_20200106_233_http_20200106_12_http_20200106_1706_http_20200106_17179873818_http_20200106_60129545782_http_20200106_3492_http_20200106_137438957183_http_20200106_3022_http_20200106_3039_http_20200106_8589937662_http_20200106_4636_http_20200106_17179889191_http_20200106_60129546916_http_20200107_266288003594_http_20200110_584115588973_http_20200106_1946_http_20200106_5423_http_20200106_22919_http_20200106_8589956365_http_20200106_128849024593_http_20200106_18_http_20200106_32_http_20200106_843_http_20200106_8589945239_http_20200106_357_http_20200106_1587_http_20200106_24009_http_20200106_1708_http_20200106_8589936509_http_20200106_17179881583_http_20200106_2761_http_20200106_35053_http_20200106_3248_http_20200106_51539626497_http_20200106_51539607673_http']
take_AP

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,...,sum,isp_date_sessioncount,total_packets_num,total_sessions_num,total_ip_num,total_country_num,srcip_set,country_set,count_country,count_ip
255,20200106_152_http_20200106_8589947030_http_202...,"[[1578239434.511962, 1578239437.425728, 157823...","[[China, China, China, China, China, China, Ch...","[[122.14.208.63, 139.199.74.166, 148.70.163.11...",[[12:/ADXEmHXq/I75cuXNFljI4DxmMVXqH46ZY1Zu8m:z...,"[[(23.1167, 113.25), (34.7725, 113.7266), (34....","[亞太電信, 亞太電信, 亞太電信, 亞太電信, 亞太電信, 亞太電信, 亞太電信, 中華電...","[http, http, http, http, http, http, http, htt...","[31897, 583, 306, 251, 226, 156, 2, 3367, 2820...",[12:/ADXEmHXq/I75cuXNFljI4DxmMVXqH46ZY1Zu8m:zA...,...,0.0,"{'亞太電信_20200106': 16433, '亞太電信_20200107': 1473...",411489900,264115,8693,128,"[222.247.121.49, 220.135.49.227, 130.180.201.9...","[Switzerland, Croatia, Saudi Arabia, Germany, ...","{'China': 144840, 'United States': 27591, 'Hon...","{'175.6.6.59': 25962, '89.248.160.221': 10757,..."


In [52]:
temp = take_AP['count_country'].values[0]
count = 0
for i,v in temp.items():
    count = count + v
144840/count , 27591/count, 17991/count , 11894/count , 11462/count

(0.548397478371164,
 0.10446585767563372,
 0.06811805463529144,
 0.04503341347519073,
 0.04339776233837533)

In [54]:
temp = take_AP['count_ip'].values[0]
count = 0
for i,v in temp.items():
    count = count + v
25962/count , 10757/count, 7689/count , 6110/count , 2033/count
# temp

(0.09829808984722564,
 0.04072847055260019,
 0.02911231849762414,
 0.02313386214338451,
 0.007697404539689151)

In [42]:
liorlili = take_AP.timestamp.values[0]
time_li = sorted(list(functools.reduce(operator.iconcat, liorlili, [])))
time_li

[1578239427.31087,
 1578239428.219373,
 1578239429.207358,
 1578239434.511962,
 1578239437.425728,
 1578239437.80298,
 1578239438.345702,
 1578239438.661876,
 1578239442.024101,
 1578239444.590476,
 1578239444.921782,
 1578239445.301088,
 1578239450.063961,
 1578239452.054208,
 1578239458.191848,
 1578239460.523634,
 1578239461.247117,
 1578239461.706728,
 1578239461.751976,
 1578239462.132985,
 1578239462.237762,
 1578239467.41652,
 1578239469.575556,
 1578239475.974706,
 1578239482.07743,
 1578239485.04883,
 1578239488.169359,
 1578239490.430152,
 1578239491.872517,
 1578239498.220286,
 1578239504.709817,
 1578239506.641326,
 1578239506.776491,
 1578239514.913572,
 1578239516.26114,
 1578239516.908293,
 1578239520.94566,
 1578239524.959808,
 1578239524.963566,
 1578239526.066168,
 1578239529.572155,
 1578239531.982262,
 1578239532.13726,
 1578239533.909301,
 1578239536.676016,
 1578239537.716887,
 1578239542.755002,
 1578239543.875425,
 1578239547.972764,
 1578239551.201314,
 1578239

In [44]:
time_li[0]

1578239427.31087

In [62]:
stri = take_AP.idx.values[0]
stri.split('_http_')[8]

'20200106_550'

In [68]:
x = Counter(take_AP['country'].values[0][8])
temp = dict(sorted(x.items(), key=lambda item: item[1],reverse=True))
count = 0
for i,v in temp.items():
    count = count + v
2039/count , 365/count, 97/count , 91/count , 36/count
# country_count

(0.7230496453900709,
 0.12943262411347517,
 0.03439716312056738,
 0.03226950354609929,
 0.01276595744680851)

In [72]:
x = Counter(take_AP['src_ip'].values[0][8])
temp = dict(sorted(x.items(), key=lambda item: item[1],reverse=True))
count = 0
for i,v in temp.items():
    count = count + v
346/count , 316/count, 225/count , 183/count , 167/count
# temp

(0.1226950354609929,
 0.11205673758865248,
 0.0797872340425532,
 0.06489361702127659,
 0.05921985815602837)

In [84]:
temp = take_AP['timestamp'].values[0][8]
sorted(temp,reverse=True)

[1578843585.830065,
 1578843442.241034,
 1578843052.075908,
 1578843040.079438,
 1578843031.075926,
 1578842802.322876,
 1578842594.022523,
 1578841337.340047,
 1578841102.0753,
 1578841096.650619,
 1578841043.07444,
 1578840251.073854,
 1578840244.076444,
 1578840238.07677,
 1578840232.074505,
 1578840226.074664,
 1578840220.592527,
 1578840077.148606,
 1578839660.075306,
 1578839653.073894,
 1578839650.65847,
 1578839647.51059,
 1578839459.597562,
 1578839253.602453,
 1578838883.938304,
 1578838635.330789,
 1578838156.418594,
 1578837909.075711,
 1578837903.074838,
 1578837897.438839,
 1578836776.074251,
 1578836449.677471,
 1578836243.074625,
 1578836237.073809,
 1578836009.512818,
 1578835500.050216,
 1578833338.615029,
 1578832784.541698,
 1578832646.080097,
 1578832633.535465,
 1578832556.072614,
 1578832543.073036,
 1578831917.343896,
 1578831442.071602,
 1578831436.071436,
 1578831430.074692,
 1578830455.071372,
 1578830449.071246,
 1578829145.071892,
 1578829139.071304,
 15788

In [76]:
take_AP.columns

Index(['idx', 'timestamp', 'country', 'src_ip', 'ssdeep', 'lalo', 'isp',
       'protocol', 'session_num', 'ssdeep_flat', 'signature', 'ssdeep_clean',
       'clusterkey_timelist', 'date_count', 'sum', 'isp_date_sessioncount',
       'total_packets_num', 'total_sessions_num', 'total_ip_num',
       'total_country_num', 'srcip_set', 'country_set', 'count_country',
       'count_ip'],
      dtype='object')

In [79]:
take_AP['clusterkey_timelist'].values[0][8]

[1578293129.36294,
 1578293129.363124,
 1578293129.474336,
 1578293131.201844,
 1578293131.202251,
 1578293131.217684,
 1578293131.227214,
 1578293131.337162,
 1578293131.38219,
 1578293132.211897,
 1578293132.212135]

In [82]:
(411489900/264115)*2820

4393546.439997729